# jupyter

> Jupyter rendering in memexplatform obsidian

In [ ]:
#| default_exp jupyter

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from memexplatform_obsidian.settings import ObsidianConfig
from enum import Enum
import urllib
from fasthtml.common import *
from monsterui.all import *
from fasthtml.jupyter import *
from collections.abc import Callable
import inspect
import ast
import json
from pathlib import Path

In [ ]:
#| export

def load_notebook(path: str):
    with open(path, "r", encoding="utf-8") as f:
        nb = json.load(f)
    return nb

In [ ]:
#| export
def extract_cells(nb):
    cells = []
    for cell in nb.get("cells", []):
        ctype = cell.get("cell_type")
        source = "".join(cell.get("source", []))
        outputs = []
        if ctype == "code":
            # extract outputs already in the notebook
            for out in cell.get("outputs", []):
                # handle different types
                if out.get("output_type") == "stream":
                    outputs.append({"type": "stream", "text": "".join(out.get("text", []))})
                elif out.get("output_type") == "execute_result":
                    data = out.get("data", {})
                    # text/plain may be present
                    text = data.get("text/plain", "")
                    outputs.append({"type": "execute_result", "text": text})
                elif out.get("output_type") == "error":
                    tb = out.get("traceback", [])
                    outputs.append({"type": "error", "text": "\n".join(tb)})
        cells.append({"cell_type": ctype, "source": source, "outputs": outputs})
    return cells

In [ ]:
#| export
_counter = 0
def _gen_id(prefix="f"):
    global _counter
    _counter += 1
    return f"{prefix}{_counter}"

def create_flippable_card(content, source_code, outputs=None, extra_cls=None):
    """
    Create a card that shows `content` (front), `source_code` toggled on "See Source",
    and optionally `outputs` toggled on "See Output".
    """
    uid = _gen_id("card")

    # Button to toggle source code
    btn_source = Button(
        "See Source",
        cls=ButtonT.primary,
        **{"data_uk_toggle": f"target: #{uid}-source"}
    )
    # Hidden Div for source
    source_div = Div(
        Pre(Code(source_code, classes="hljs language-python")),
        id=f"{uid}-source",
        hidden=True,
        cls="mt-2 p-2 bg-gray-100 rounded"
    )

    children = [btn_source, content, source_div]

    if outputs:
        btn_outputs = Button(
            "See Output",
            cls=ButtonT.secondary,
            **{"data_uk_toggle": f"target: #{uid}-output"}
        )
        output_div = Div(
            *[Pre(o) for o in outputs],
            id=f"{uid}-output",
            hidden=True,
            cls="mt-2 p-2 bg-gray-100 rounded text-gray-800"
        )
        children.insert(1, btn_outputs)  # place output button before content maybe
        children.append(output_div)

    card = Card(*children, cls=("my-6 " + extra_cls) if extra_cls else "my-6")
    return card

In [ ]:
path = '/Users/rahul1.saraf/rahuketu/programming/notesobs/pages/ThinkBayes.01_Probability.ipynb'
nb_content = json.loads(Path(path).read_text()); nb_content


{'cells': [{'cell_type': 'raw',
   'id': '9e58714a',
   'metadata': {'lines_to_next_cell': 0},
   'source': ['---\n',
    'title: ThinkBayes.01_Probability\n',
    'collection:\n',
    '  - "[[Think Bayes]]"\n',
    'logseq_url: logseq://graph/notesobs?page=ThinkBayes.01_Probability\n',
    '---']},
  {'cell_type': 'markdown',
   'id': '84cc2de4',
   'metadata': {},
   'source': ['Probability is defined as a fraction of a finite set. The word "random" in "random probability" means everyone has the same chance of being chosen, but the chance is probably close to 2%. ']},
  {'cell_type': 'code',
   'execution_count': 2,
   'id': '0dfa009e',
   'metadata': {},
   'outputs': [],
   'source': ['import pandas as pd\n',
    'import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import random']},
  {'cell_type': 'code',
   'execution_count': 4,
   'id': '9a9cfec4',
   'metadata': {},
   'outputs': [{'data': {'text/html': ['<div>\n',
       '<style scoped>\n',
       '    .datafr

In [ ]:
#| export
def render_nb(nb_path: str):
    nb = load_notebook(nb_path)
    cells = extract_cells(nb)
    blocks = []
    for cell in cells:
        if cell["cell_type"] == "markdown":
            if cell["source"].strip():
                md_html = render_md(cell["source"])
                blocks.append(Card(
                    Safe(md_html),
                    cls="my-4 shadow"
                ))
        elif cell["cell_type"] == "code":
            if cell["source"].strip():
                # content: show code front
                content = Pre(Code(cell["source"], classes="hljs language-python"))
                # prepare outputs list of plain strings
                outs = [o["text"] for o in cell.get("outputs", []) if o.get("text")]
                blocks.append(
                    create_flippable_card(
                        content=content,
                        source_code=cell["source"],
                        outputs=outs,           # now allowed
                        extra_cls="border border-gray-300"
                    )
                )
    return Container(*blocks, cls="p-6 space-y-6")

In [ ]:
render_nb(path)

```html
<div class="uk-container p-6 space-y-6">
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><p class="text-lg leading-relaxed mb-6">Probability is defined as a fraction of a finite set. The word "random" in "random probability" means everyone has the same chance of being chosen, but the chance is probably close to 2%.</p>
</div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card1-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><pre><code classes="hljs language-python">import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random</code></pre>      <div hidden id="card1-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random</code></pre>      </div>
    </div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card2-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><button data-uk-toggle="target: #card2-output" type="submit" class="uk-btn uk-btn-secondary">See Output</button><pre><code classes="hljs language-python">df = pd.read_csv(&#x27;../_data/gss_bayes.csv&#x27;); df.head()</code></pre>      <div hidden id="card2-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">df = pd.read_csv(&#x27;../_data/gss_bayes.csv&#x27;); df.head()</code></pre>      </div>
      <div hidden id="card2-output" class="mt-2 p-2 bg-gray-100 rounded text-gray-800">
<pre>['   caseid  year   age  sex  polviews  partyid  indus10\n', '0       1  1974  21.0    1       4.0      2.0   4970.0\n', '1       2  1974  41.0    1       5.0      0.0   9160.0\n', '2       5  1974  58.0    2       6.0      1.0   2670.0\n', '3       6  1974  30.0    1       5.0      4.0   6870.0\n', '4       7  1974  48.0    1       5.0      4.0   7860.0']</pre>      </div>
    </div>
  </div>
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><div><h2 class="uk-h2 text-3xl font-bold mt-10 mb-5">Data Key</h2>
<ul class="uk-list uk-list-bullet space-y-2 mb-6 ml-6 text-lg">
<li class="leading-relaxed">
<p class="text-lg leading-relaxed mb-6">caseid: Respondent identifier.</p>
</li>
<li class="leading-relaxed">
<p class="text-lg leading-relaxed mb-6">year: Year when the respondent was surveyed.</p>
</li>
<li class="leading-relaxed">
<p class="text-lg leading-relaxed mb-6">age: Respondent’s age when surveyed.</p>
</li>
<li class="leading-relaxed">
<p class="text-lg leading-relaxed mb-6">sex: Male or female.</p>
</li>
<li class="leading-relaxed">
<p class="text-lg leading-relaxed mb-6">polviews: Political views on a range from liberal to conservative.</p>
</li>
<li class="leading-relaxed">
<p class="text-lg leading-relaxed mb-6">partyid: Political party affiliation, Democrat, Independent, or Republican.</p>
</li>
<li class="leading-relaxed">
<p class="text-lg leading-relaxed mb-6">indus10: Code for the industry the respondent works in</p>
</li>
</ul>
<p class="text-lg leading-relaxed mb-6">“Banking and related activities” is 6870</p>
<p class="text-lg leading-relaxed mb-6">Political views :-</p>
<ul class="uk-list uk-list-bullet space-y-2 mb-6 ml-6 text-lg">
<li class="leading-relaxed">1	Extremely liberal</li>
<li class="leading-relaxed">2	Liberal</li>
<li class="leading-relaxed">3	Slightly liberal</li>
<li class="leading-relaxed">4	Moderate</li>
<li class="leading-relaxed">5	Slightly conservative</li>
<li class="leading-relaxed">6	Conservative</li>
<li class="leading-relaxed">7	Extremely conservative</li>
</ul>
<p class="text-lg leading-relaxed mb-6">Party id</p>
<ul class="uk-list uk-list-bullet space-y-2 mb-6 ml-6 text-lg">
<li class="leading-relaxed">0 Strong democrat</li>
<li class="leading-relaxed">1 Not strong democrat</li>
<li class="leading-relaxed">2 Independent, near democrat</li>
<li class="leading-relaxed">3 Independent</li>
<li class="leading-relaxed">4 Independent, near republican</li>
<li class="leading-relaxed">5 Not strong republican</li>
<li class="leading-relaxed">6 Strong republican</li>
<li class="leading-relaxed">7 Other party</li>
</ul>
</div></div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card3-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><button data-uk-toggle="target: #card3-output" type="submit" class="uk-btn uk-btn-secondary">See Output</button><pre><code classes="hljs language-python">banker = (df.indus10 == 6870);  np.round(banker.mean()*100, 1), banker.sum() #banker,
female = (df.sex == 2); np.round(female.mean()*100, 1), female.sum() #female,
liberal = (df.polviews &lt;= 3); np.round(liberal.mean()*100, 1), liberal.sum() #liberal,
conservative = (df.polviews &gt;= 5); np.round(conservative.mean()*100, 1), conservative.sum() #conservative,
democrat = (df.partyid &lt;=1 ); np.round(democrat.mean()*100, 1), democrat.sum() #democrat,

def prob(A):return A.mean()

prob(banker), prob(female), prob(liberal), prob(democrat)
</code></pre>      <div hidden id="card3-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">banker = (df.indus10 == 6870);  np.round(banker.mean()*100, 1), banker.sum() #banker,
female = (df.sex == 2); np.round(female.mean()*100, 1), female.sum() #female,
liberal = (df.polviews &lt;= 3); np.round(liberal.mean()*100, 1), liberal.sum() #liberal,
conservative = (df.polviews &gt;= 5); np.round(conservative.mean()*100, 1), conservative.sum() #conservative,
democrat = (df.partyid &lt;=1 ); np.round(democrat.mean()*100, 1), democrat.sum() #democrat,

def prob(A):return A.mean()

prob(banker), prob(female), prob(liberal), prob(democrat)
</code></pre>      </div>
      <div hidden id="card3-output" class="mt-2 p-2 bg-gray-100 rounded text-gray-800">
<pre>['(np.float64(0.014769730168391155),\n', ' np.float64(0.5378575776019476),\n', ' np.float64(0.27374721038750255),\n', ' np.float64(0.3662609048488537))']</pre>      </div>
    </div>
  </div>
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><div><h1 class="uk-h1 text-4xl font-bold mt-12 mb-6">Conjuction</h1>
<blockquote class="uk-blockquote pl-4 border-l-4 border-primary italic mb-6">
<p class="text-lg leading-relaxed mb-6">“Conjunction” is another name for the logical and operation. If you have two propositions, A and B, the conjunction A and B is True if both A and B are True, and False otherwise</p>
</blockquote>
</div></div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card4-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><button data-uk-toggle="target: #card4-output" type="submit" class="uk-btn uk-btn-secondary">See Output</button><pre><code classes="hljs language-python">prob(banker &amp; democrat), prob(democrat &amp; banker) # Commutative</code></pre>      <div hidden id="card4-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">prob(banker &amp; democrat), prob(democrat &amp; banker) # Commutative</code></pre>      </div>
      <div hidden id="card4-output" class="mt-2 p-2 bg-gray-100 rounded text-gray-800">
<pre>['(np.float64(0.004686548995739501), np.float64(0.004686548995739501))']</pre>      </div>
    </div>
  </div>
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><div><h1 class="uk-h1 text-4xl font-bold mt-12 mb-6">Conditional Probability</h1>
<blockquote class="uk-blockquote pl-4 border-l-4 border-primary italic mb-6">
<p class="text-lg leading-relaxed mb-6">Probability that depends on a condition
It is calculated as a 2 step process</p>
<ul class="uk-list uk-list-bullet space-y-2 mb-6 ml-6 text-lg">
<li class="leading-relaxed">Select all who are liberal</li>
<li class="leading-relaxed">Compute fraction that are liberal</li>
</ul>
</blockquote>
</div></div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card5-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><button data-uk-toggle="target: #card5-output" type="submit" class="uk-btn uk-btn-secondary">See Output</button><pre><code classes="hljs language-python">selected = democrat[liberal]; prob(selected), prob(democrat &amp; liberal) # not same we are taking fraction of a subset when calculating condidtional probabilities</code></pre>      <div hidden id="card5-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">selected = democrat[liberal]; prob(selected), prob(democrat &amp; liberal) # not same we are taking fraction of a subset when calculating condidtional probabilities</code></pre>      </div>
      <div hidden id="card5-output" class="mt-2 p-2 bg-gray-100 rounded text-gray-800">
<pre>['(np.float64(0.5206403320240125), np.float64(0.1425238385067965))']</pre>      </div>
    </div>
  </div>
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><blockquote class="uk-blockquote pl-4 border-l-4 border-primary italic mb-6">
<p class="text-lg leading-relaxed mb-6">Let’s try the second example, “What is the probability that a respondent is female, given that they are a banker?” We can interpret that to mean, “Of all respondents who are bankers, what fraction are female?”</p>
</blockquote>
</div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card6-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><button data-uk-toggle="target: #card6-output" type="submit" class="uk-btn uk-btn-secondary">See Output</button><pre><code classes="hljs language-python">selected = female[banker]; prob(selected), prob(female &amp; banker) # same as above</code></pre>      <div hidden id="card6-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">selected = female[banker]; prob(selected), prob(female &amp; banker) # same as above</code></pre>      </div>
      <div hidden id="card6-output" class="mt-2 p-2 bg-gray-100 rounded text-gray-800">
<pre>['(np.float64(0.7706043956043956), np.float64(0.011381618989653074))']</pre>      </div>
    </div>
  </div>
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><blockquote class="uk-blockquote pl-4 border-l-4 border-primary italic mb-6">
<p class="text-lg leading-relaxed mb-6">Define conditional to take two Boolean series, proposition and given, and compute the conditional probability of proposition conditioned on given</p>
</blockquote>
</div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card7-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><pre><code classes="hljs language-python">def conditional(proposition, given):
    &quot;&quot;&quot;Probability of A conditioned on given.&quot;&quot;&quot;
    return prob(proposition[given])
</code></pre>      <div hidden id="card7-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">def conditional(proposition, given):
    &quot;&quot;&quot;Probability of A conditioned on given.&quot;&quot;&quot;
    return prob(proposition[given])
</code></pre>      </div>
    </div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card8-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><button data-uk-toggle="target: #card8-output" type="submit" class="uk-btn uk-btn-secondary">See Output</button><pre><code classes="hljs language-python">conditional(liberal, given=female) # To be read as About 28% of female respondents are liberal.</code></pre>      <div hidden id="card8-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">conditional(liberal, given=female) # To be read as About 28% of female respondents are liberal.</code></pre>      </div>
      <div hidden id="card8-output" class="mt-2 p-2 bg-gray-100 rounded text-gray-800">
<pre>['np.float64(0.27581004111500884)']</pre>      </div>
    </div>
  </div>
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><h2 class="uk-h2 text-3xl font-bold mt-10 mb-5">Conditional probability is not commutative</h2>
</div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card9-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><button data-uk-toggle="target: #card9-output" type="submit" class="uk-btn uk-btn-secondary">See Output</button><pre><code classes="hljs language-python">conditional(female, given=banker) </code></pre>      <div hidden id="card9-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">conditional(female, given=banker) </code></pre>      </div>
      <div hidden id="card9-output" class="mt-2 p-2 bg-gray-100 rounded text-gray-800">
<pre>['np.float64(0.7706043956043956)']</pre>      </div>
    </div>
  </div>
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><blockquote class="uk-blockquote pl-4 border-l-4 border-primary italic mb-6">
<p class="text-lg leading-relaxed mb-6">To be read as About 77% of bankers are female</p>
</blockquote>
</div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card10-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><button data-uk-toggle="target: #card10-output" type="submit" class="uk-btn uk-btn-secondary">See Output</button><pre><code classes="hljs language-python">conditional(banker, given=female) </code></pre>      <div hidden id="card10-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">conditional(banker, given=female) </code></pre>      </div>
      <div hidden id="card10-output" class="mt-2 p-2 bg-gray-100 rounded text-gray-800">
<pre>['np.float64(0.02116102749801969)']</pre>      </div>
    </div>
  </div>
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><blockquote class="uk-blockquote pl-4 border-l-4 border-primary italic mb-6">
<p class="text-lg leading-relaxed mb-6">About 2% of females are banker</p>
</blockquote>
</div>
  </div>
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><h2 class="uk-h2 text-3xl font-bold mt-10 mb-5">Condition and Conjuction</h2>
</div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card11-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><button data-uk-toggle="target: #card11-output" type="submit" class="uk-btn uk-btn-secondary">See Output</button><pre><code classes="hljs language-python">conditional(female, given=liberal &amp; democrat)</code></pre>      <div hidden id="card11-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">conditional(female, given=liberal &amp; democrat)</code></pre>      </div>
      <div hidden id="card11-output" class="mt-2 p-2 bg-gray-100 rounded text-gray-800">
<pre>['np.float64(0.576085409252669)']</pre>      </div>
    </div>
  </div>
  <div class="uk-card my-4 shadow">
    <div class="uk-card-body space-y-6"><blockquote class="uk-blockquote pl-4 border-l-4 border-primary italic mb-6">
<p class="text-lg leading-relaxed mb-6">Probability a respondent is female given they are liberal and democrat</p>
</blockquote>
</div>
  </div>
  <div class="uk-card my-6 border border-gray-300">
    <div class="uk-card-body space-y-6">
<button data-uk-toggle="target: #card12-source" type="submit" class="uk-btn uk-btn-primary">See Source</button><button data-uk-toggle="target: #card12-output" type="submit" class="uk-btn uk-btn-secondary">See Output</button><pre><code classes="hljs language-python">print(&quot;Hello, world&quot;)</code></pre>      <div hidden id="card12-source" class="mt-2 p-2 bg-gray-100 rounded">
<pre><code classes="hljs language-python">print(&quot;Hello, world&quot;)</code></pre>      </div>
      <div hidden id="card12-output" class="mt-2 p-2 bg-gray-100 rounded text-gray-800">
<pre>Hello, world
</pre>      </div>
    </div>
  </div>
</div>

```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()